# 3D Interpolation with Scipy

The hall probe measurement simulation software is hamstrung by the sparseness of the available field grid data.  The Mau10 and GA05 field grids are given in Cartesian coordinates, with 25 mm spacing on the X, Y, and Z axes.  The hall probe measurement device will work most naturally in cylindrical coordinates.  In order to sample regular cylindrically symmetric data from the field grids, one must either cherry pick values that coincide with the 25x25x25 mm spacing, or generate data through interpolation.

The current interpolation method employed in the Mu2E organization for doing this is a linear 3D method.  This leads to can lead to fairly large errors in regions of large magnetic gradient, which would be unsuitable for further analysis.  We will attempt to use a more robust interpolate method and compare those results with random field data (provided in a separate file for validation purposes).

In [212]:
from mu2e.datafileprod import DataFileMaker
import numpy as np

In [2]:
df = DataFileMaker('../datafiles/Mau10/Standard_Maps/Mu2e_DSMap',use_pickle = True).data_frame

In [3]:
df.head()

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
0,-1200.0,-1200.0,3071.0,0.129282,0.132041,0.044331,1697.056275,-2.356194,-0.001951,-0.184783
1,-1200.0,-1200.0,3096.0,0.132108,0.134881,0.041163,1697.056275,-2.356194,-0.001960,-0.188790
2,-1200.0,-1200.0,3121.0,0.134887,0.137672,0.037731,1697.056275,-2.356194,-0.001969,-0.192728
3,-1200.0,-1200.0,3146.0,0.137602,0.140399,0.034029,1697.056275,-2.356194,-0.001977,-0.196576
4,-1200.0,-1200.0,3171.0,0.140237,0.143045,0.030050,1697.056275,-2.356194,-0.001985,-0.200311


In [68]:
df_rand = DataFileMaker('../datafiles/Mau10/Standard_Maps/Mu2e_DSMap_rand1mil',use_pickle = True).data_frame

In [69]:
df_rand.head()

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
0,-1007.9996,-0.708404,12190.6920,-0.005847,-0.000004,0.993192,1007.999849,-3.140890,-1.790000e-07,0.005847
1,-1007.9966,-1.116473,7536.1132,-0.564179,-0.000629,1.455297,1007.997218,-3.140485,4.159000e-06,0.564180
2,-1007.9943,-1.397842,5638.5763,0.147173,0.000207,1.172049,1007.995269,-3.140206,-3.246000e-06,-0.147173
3,-1007.9935,-1.022141,7750.8171,-0.221387,-0.000226,0.703601,1007.994018,-3.140579,1.061000e-06,0.221387
4,-1007.9879,-1.502013,8048.6632,0.059701,0.000091,1.010066,1007.989019,-3.140103,-1.725000e-06,-0.059701


In [72]:
df_rand.Y.max()

999.95324000000005

We have our dataframes queued up.  We will attempt to interpolate a regular grid from `df`, and test values from `df_rand` to determine our accuracy.  Let's try to use scipy.interpolate.Rbf in a simple case, first:

In [6]:
from scipy.interpolate import Rbf

In [7]:
x, y, z, d = np.random.rand(4, 50)
rbfi = Rbf(x, y, z, d)
xi = yi = zi = np.linspace(0, 1, 20)
di = rbfi(xi, yi, zi)

In [8]:
print di

[ 0.38180703  0.27407509  0.18086105  0.12909194  0.16130581  0.30530448
  0.52734052  0.73946123  0.85943045  0.85721002  0.76313443  0.63024096
  0.48864523  0.34932557  0.22487432  0.1269399   0.05889031  0.01766057
 -0.00201629 -0.0051106 ]


In [91]:
df_subset = df.query('-200<=X<=200 and -200<=Y<=200 and 8000<Z<8200')

In [92]:
df_subset.head()

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
2042518,-200.0,-200.0,8021.0,-0.010097,-0.009709,1.046913,282.842712,-2.356194,-0.000274,0.014005
2042519,-200.0,-200.0,8046.0,-0.009340,-0.008925,1.044676,282.842712,-2.356194,-0.000293,0.012915
2042520,-200.0,-200.0,8071.0,-0.008612,-0.008171,1.042650,282.842712,-2.356194,-0.000312,0.011868
2042521,-200.0,-200.0,8096.0,-0.007917,-0.007452,1.040825,282.842712,-2.356194,-0.000329,0.010868
2042522,-200.0,-200.0,8121.0,-0.007260,-0.006772,1.039187,282.842712,-2.356194,-0.000345,0.009922


In [205]:
x = df_subset.X
y = df_subset.Y
z = df_subset.Z
r = df_subset.R
phi = df_subset.Phi
bz = df_subset.Bz
br = df_subset.Br
bphi = df_subset.Bphi



In [224]:
def cylindrical_norm(x1,x2):
    return np.sqrt(
        (x1[0,:]*np.cos(x1[1,:])-x2[0,:]*np.cos(x2[1,:]))**2 +
        (x1[0,:]*np.sin(x1[1,:])-x2[0,:]*np.sin(x2[1,:]))**2 +
        (x1[2,:]-x2[2,:])**2)

In [405]:
df_rand_subset = df_rand.query('-150<=X<=150 and -150<=Y<=150 and 8050<Z<8150')

In [406]:
rbf_bz = Rbf(r, phi, z, bz,function='quintic', norm=cylindrical_norm)

In [407]:
i_bz = rbf_bz(df_rand_subset.R, df_rand_subset.Phi, df_rand_subset.Z)
t_bz = df_rand_subset.Bz

In [408]:
res =abs(i_bz-t_bz)

In [409]:
max(res)

2.0260039196973167e-05

In [416]:
rbf_br = Rbf(r, phi, z, br,function='gaussian', norm=cylindrical_norm)
i_br = rbf_br(df_rand_subset.R, df_rand_subset.Phi, df_rand_subset.Z)
t_br = df_rand_subset.Br
res =abs(i_br-t_br)
max(res)

0.008951273414076439

In [411]:
rbf_bphi = Rbf(r, phi, z, bphi,function='quintic', norm=cylindrical_norm)
i_bphi = rbf_bphi(df_rand_subset.R, df_rand_subset.Phi, df_rand_subset.Z)
t_bphi = df_rand_subset.Bphi
res =abs(i_bphi-t_bphi)
max(res)

4.1370253890748261e-05